In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

# --- Load and prepare data ---
record_id = '418'
X = np.load(f'processed_data/{record_id}_segments.npy').astype(np.float32)
y = np.load(f'processed_data/{record_id}_labels.npy').astype(np.int32)

# Optional: Normalize signals
# X = (X - np.mean(X)) / np.std(X)

# --- Train/Validation Split ---
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# --- CNN Model for Binary Classification ---
model = models.Sequential([
    layers.Input(shape=(2000, 2)),

    layers.Conv1D(16, kernel_size=7, activation='relu'),
    layers.MaxPooling1D(pool_size=2),

    layers.Conv1D(32, kernel_size=5, activation='relu'),
    layers.MaxPooling1D(pool_size=2),

    layers.Conv1D(64, kernel_size=3, activation='relu'),
    layers.GlobalAveragePooling1D(),

    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),  # helps prevent overfitting

    layers.Dense(1, activation='sigmoid')  # binary output
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)               │ (None, 1994, 16)       │           240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 997, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 993, 32)        │         2,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 496, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 494, 64)        │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ (None, 64)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,153 (43.57 KB)

 Trainable params: 11,153 (43.57 KB)

 Non-trainable params: 0 (0.00 B)

In [3]:

# --- Train Model ---
history = model.fit(X_train, y_train, epochs=15, batch_size=32,
                    validation_data=(X_val, y_val))


Epoch 1/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.4844 - loss: 0.6931 - val_accuracy: 0.5094 - val_loss: 0.6928
Epoch 2/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.5590 - loss: 0.6912 - val_accuracy: 0.5094 - val_loss: 0.6929
Epoch 3/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.5418 - loss: 0.6928 - val_accuracy: 0.5094 - val_loss: 0.6928
Epoch 4/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.5505 - loss: 0.6907 - val_accuracy: 0.5094 - val_loss: 0.6926
Epoch 5/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.5449 - loss: 0.6893 - val_accuracy: 0.5094 - val_loss: 0.6925
Epoch 6/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5334 - loss: 0.6918 - val_accuracy: 0.5094 - val_loss: 0.6923
Epoch 7/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.5178 - loss: 0.6916 - val_accuracy: 0.5094 - val_loss: 0.6922
Epoch 8/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.5747 - loss: 0.6895 - val_accuracy: 0.5094 - val_loss: 0.6921


In [ ]:

# --- Save the Model and Label Map ---
model.save(f'models/ecg_cnn_{record_id}.h5')
np.save(f'models/{record_id}_label_classes.npy', label_encoder.classes_)
